# How-To Guide: Memory

This notebook shows how to use Steamship-backed memory to persist state in your LangChain applications.

For demonstration purposes, we will use a temporary workspace. In production environments, a permanent workspace would be used to provide persistence, as well as user-scoping.

In [1]:
from langchain import LLMChain, PromptTemplate
from steamship import Steamship
from steamship_langchain.llms import OpenAI
import logging

logging.disable(logging.CRITICAL)  # disable warning messages to declutter

In [2]:
# Establish a prompt for use

template = """You are a chatbot having a conversation with a human.

{history}
Human: {human_input}
Chatbot:"""

chat_prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)

In [3]:
# Establish a series of inputs for the converation (as an example)

user_messages = [
    "Do you prefer glazed doughnuts or old-fashioned doughnuts?",
    "How does that compare to chocolate eclairs in your opinion?",
    "Would you prefer that or a croissant for breakfast?",
    "What is the best place to get some near where you live?",
]

In [4]:
# use persistent chat memory
from steamship_langchain.memory import ChatMessageHistory
from langchain.memory import ConversationBufferMemory

with Steamship.temporary_workspace() as client:
    saved_history = ChatMessageHistory(client=client, key="example-user-chat-1234")
    memory = ConversationBufferMemory(chat_memory=saved_history)

    chatgpt = LLMChain(
        llm=OpenAI(client=client, temperature=0),
        prompt=chat_prompt,
        memory=memory,
    )

    for message in user_messages:
        print(f"Human: {message}")
        print(f"AI: {chatgpt.predict(human_input=message)}")

    print("\n\n--- Persistent Transcript From Memory ---")

    print(f"{memory.load_memory_variables(inputs={}).get('history', '')}")

Human: Do you prefer glazed doughnuts or old-fashioned doughnuts?
AI:  That's a tough one! I think I would have to go with glazed doughnuts. They just have a special something that old-fashioned doughnuts don't have.
Human: How does that compare to chocolate eclairs in your opinion?
AI:  I think chocolate eclairs are delicious, but they don't quite compare to the sweetness and texture of glazed doughnuts.
Human: Would you prefer that or a croissant for breakfast?
AI:  I think I would prefer a croissant for breakfast. Croissants are light and fluffy, and they pair well with a cup of coffee.
Human: What is the best place to get some near where you live?
AI:  The best place to get croissants near me is a local bakery called Sweet Tooth Bakery. They have the freshest and most delicious croissants around!


--- Persistent Transcript From Memory ---
Human: Do you prefer glazed doughnuts or old-fashioned doughnuts?
AI:  That's a tough one! I think I would have to go with glazed doughnuts. The

In [6]:
# use persistent chat memory
from steamship_langchain.memory import ChatMessageHistory
from langchain.memory import ConversationBufferWindowMemory

with Steamship.temporary_workspace() as client:
    saved_history = ChatMessageHistory(client=client, key="example-user-chat-1234")
    memory = ConversationBufferWindowMemory(chat_memory=saved_history, k=2)

    chatgpt = LLMChain(
        llm=OpenAI(client=client, temperature=0),
        prompt=chat_prompt,
        memory=memory,
    )

    for message in user_messages:
        print(f"Human: {message}")
        print(f"AI: {chatgpt.predict(human_input=message)}")

    print("\n\n--- Persistent Transcript From Memory (k=2) ---")

    print(f"{memory.load_memory_variables(inputs={}).get('history', '')}")

Human: Do you prefer glazed doughnuts or old-fashioned doughnuts?
AI:  That's a tough one! I think I would have to go with glazed doughnuts. They just have a special something that old-fashioned doughnuts don't have.
Human: How does that compare to chocolate eclairs in your opinion?
AI:  I think chocolate eclairs are delicious, but they don't quite compare to the sweetness and texture of glazed doughnuts.
Human: Would you prefer that or a croissant for breakfast?
AI:  I think I would prefer a croissant for breakfast. Croissants are light and fluffy, and they pair well with a cup of coffee.
Human: What is the best place to get some near where you live?
AI:  The best place to get croissants near me is a local bakery called Sweet Treats. They make the most delicious croissants in town!


--- Persistent Transcript From Memory (k=2) ---
Human: Would you prefer that or a croissant for breakfast?
AI:  I think I would prefer a croissant for breakfast. Croissants are light and fluffy, and they 